In [1]:
import cv2
import numpy as np
import sys


## Utilities

In [2]:
def frame_differencing(prev_frame, curr_frame):
    """Calculate frame difference."""
    diff = cv2.absdiff(curr_frame, prev_frame)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)
    return thresh

def skin_detection(frame):
    """Detect skin in the given frame."""
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_skin = np.array([0, 20, 30], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    return mask

def motion_energy(frames):
    """Calculate motion energy across a list of frames."""
    motion_history = np.zeros(frames[0].shape[:2], dtype=np.float32)
    for frame in frames:
        motion_history = cv2.motempl.updateMotionHistory(frame, motion_history, 1, 5)
    return motion_history

def accumulate_motion_energy(frames):
    """Accumulate motion energy from a list of binary difference frames."""
    # Assuming frames are binary (0 or 255), where 255 indicates motion
    # Initialize an accumulator frame with zeros
    acc = np.zeros_like(frames[0], dtype=np.float32)
    
    # Accumulate frames by adding
    for frame in frames:
        # Convert frame to float to prevent data type overflow
        acc = acc + frame.astype(np.float32)
    
    # Normalize the accumulated image to the range [0, 255]
    acc = np.clip((acc / len(frames)), 0, 255).astype(np.uint8)
    
    return acc

def find_centroid(binary_image):
    """Find the centroid of binary objects in the image."""
    # Calculate moments of the binary image
    moments = cv2.moments(binary_image)
    
    # Use the moments to calculate the centroid (cx, cy)
    if moments["m00"] != 0:
        cx = int(moments["m10"] / moments["m00"])
        cy = int(moments["m01"] / moments["m00"])
    else:
        # Default to the center of the image if no mass found
        cx, cy = binary_image.shape[1] // 2, binary_image.shape[0] // 2
    
    return (cx, cy)

def morph_image(image, morph, kernel_size):
    def erosion(image_cv, kernel_size=4):
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
        image_cv_eroded = cv2.erode(image_cv, kernel)
        return image_cv_eroded

    def dilate(image_cv, kernel_size=4):
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
        image_cv_dilated = cv2.dilate(image_cv, kernel)
        return image_cv_dilated

    if morph == 'dilate':
        image_morphed = dilate(image, kernel_size=kernel_size)
    elif morph == 'erode':
        image_morphed = erosion(image, kernel_size=kernel_size)
    else:
        raise NotImplementedError("Morphological operation not implemented.")
    
    return image_morphed

## Main

In [3]:
def main():
    cap = cv2.VideoCapture(0)  # Use 0 for webcam

    if not cap.isOpened():
        print("Cannot open camera")
        sys.exit()

    ret, prev_frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        return

    # Calculate new dimensions for cropping (50% of the original size)
    h, w = prev_frame.shape[:2]
    new_w, new_h = w // 2, h // 2

    # Calculate the offset to crop the frame to its center
    x_offset = (w - new_w) // 2
    y_offset = (h - new_h) // 2

    # Apply cropping to reduce the frame size by 50% (center of the frame)
    prev_frame = prev_frame[y_offset:y_offset+new_h, x_offset:x_offset+new_w]
    prev_frame = cv2.flip(prev_frame, 1)  # Flip frame
    frames_for_motion = []
    circularity_values = []  # List to store circularity values
    max_inertia_values = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Apply the same cropping to the new frame
        frame = frame[y_offset:y_offset+new_h, x_offset:x_offset+new_w]
        frame = cv2.flip(frame, 1)  # Flip frame for a mirror effect

        # Proceed with your existing processing
        frame_diff = frame_differencing(prev_frame, frame)
        
        eroded = morph_image(frame, 'erode', 4)
        refined = morph_image(eroded, 'dilate', 4)
        skin_mask = skin_detection(refined)

        # Convert binary mask to a 3-channel BGR image to show colored centroid and boundary box
        skin_mask_color = cv2.cvtColor(skin_mask, cv2.COLOR_GRAY2BGR)

        # Existing code to calculate the centroid
        centroid = find_centroid(skin_mask)
        
        # Draw a red dot at the centroid on the skin mask
        # Draw the centroid
        cv2.circle(skin_mask_color, centroid, 15, (0, 0, 255), -1)  # Red centroid

        # Coordinates for the text, slightly above the centroid
        text_x, text_y = centroid[0], centroid[1] - 20

        # Draw contours (boundary box) on the color mask
        contours, _ = cv2.findContours(skin_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(skin_mask_color, contours, -1, (255, 0, 0), 2)  # Blue contours
        # for cnt in contours:
        #     M = cv2.moments(cnt)
        #     if M["m00"] != 0:
        #         cx = int(M["m10"] / M["m00"])
        #         cy = int(M["m01"] / M["m00"])
        #         area = M["m00"]
        #         perimeter = cv2.arcLength(cnt, True)
        #         circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter != 0 else 0
                
        #         # For inertia, assuming a simple case where the object is approximated as a rectangle:
        #         mu20 = M["mu20"] / M["m00"]
        #         mu02 = M["mu02"] / M["m00"]
        #         inertia = mu20 + mu02  # Simplified inertia calculation
        #         text = f'Circularity: {circularity:.2f}, Inertia: {inertia:.2f}'
        #         cv2.putText(skin_mask_color, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
        #                     0.7, (0, 0, 255), 2, cv2.LINE_AA)
        #         print(f"Circularity: {circularity}, Inertia: {inertia}")
        max_area = 0
        max_circularity = 0
        max_inertia = 0
        for cnt in contours:
            M = cv2.moments(cnt)
            if M["m00"] > max_area:  # Update for the largest contour
                max_area = M["m00"]
                perimeter = cv2.arcLength(cnt, True)
                # max_circularity = (4 * np.pi * max_area) / (perimeter ** 2) if perimeter != 0 else 0
                mu20 = M["mu20"] / M["m00"]
                mu02 = M["mu02"] / M["m00"]
                max_inertia = mu20 + mu02 
                perimeter = cv2.arcLength(cnt, True)
                circularity = (4 * np.pi * max_area) / (perimeter ** 2) if perimeter != 0 else 0
                

        if max_area > 0:  # Check to ensure at least one contour was found
            # Update circularity values list
            circularity_values.append(circularity)
            max_inertia_values.append(max_inertia)
            if len(circularity_values) > 10:  # Keep only the last 10 values
                circularity_values.pop(0)
                max_inertia_values.pop(0)

            # Calculate the average circularity if there are enough values
            if len(circularity_values) == 10:
                average_circularity = sum(circularity_values) / len(circularity_values)
                average_inertia = sum(max_inertia_values) / len(max_inertia_values)
                text = f'Avg Circularity (10 frames): {average_circularity:.2f} Avg Inertia (10 frames): {average_inertia:.2f}'
                cv2.putText(skin_mask_color, text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                            0.7, (0, 0, 255), 2, cv2.LINE_AA)
        # Add the text label above the centroid
        # cv2.putText(skin_mask_color, 'Centroid', (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        # if max_area > 0:  # Ensure there is at least one contour
        #     text = f'Largest Contour - Circularity: {max_circularity:.2f}, Inertia: {max_inertia:.2f}'
        #     cv2.putText(skin_mask_color, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
        #                 0.7, (0, 0, 255), 2, cv2.LINE_AA)

        # contours, _ = cv2.findContours(skin_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # cv2.drawContours(skin_mask, contours, -1, (255, 0, 0), 2)  # Draw contours on the skin mask
        # cv2.imshow('Skin Mask', skin_mask)
        cv2.imshow('Skin Mask with Color', skin_mask_color)


        

        # Combine frame difference and skin detection for better results
        combined = cv2.bitwise_and(frame_diff, frame_diff, mask=skin_mask)

        cv2.imshow('Frame Difference', combined)

        frames_for_motion.append(combined)
        if len(frames_for_motion) > 5:  # Keeping the last 5 frames
            frames_for_motion.pop(0)

        # Calculate and visualize accumulated motion energy (optional)
        if len(frames_for_motion) >= 5:  # Ensure there are enough frames
            accumulated_me = accumulate_motion_energy(frames_for_motion)
            cv2.imshow('Accumulated Motion Energy', accumulated_me)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        prev_frame = frame

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


2024-02-11 00:44:18.451 python[2984:24429466] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
